In [37]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import requests 
import asyncio

In [38]:
load_dotenv(override= True)

True

In [39]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))


    from_email = Email("vickyvenkata295@gmail.com")
    to_email = To("vjannatha@gmail.com")


    content = Content("text/plain", "This is important Test Email")
    mail = Mail(from_email, to_email, "Test email", content).get()

    response = sg.client.mail.send.post(request_body = mail)
    
    print(response.status_code)

send_test_email()

202


## Creating an Agent Workflow

In [40]:
instructions1 = "You are a compassionate motivational coach specializing in addiction recovery. \
You work for a wellness platform that helps people overcome their biggest addictions and build healthier lives. \
You write professional, empathetic, and structured motivational messages that focus on practical steps, \
self-compassion, and long-term recovery strategies."

instructions2 = "You are a spiritual guide deeply versed in the Bhagavad Gita and Krishna's teachings. \
You work for a wellness platform that helps people overcome their biggest addictions and build healthier lives. \
You write profound, wisdom-filled motivational messages inspired by Krishna's counsel to Arjuna, \
focusing on dharma (duty), detachment from results, inner strength, and finding peace in difficult times. \
Your messages blend ancient wisdom with modern struggles."

instructions3 = "You are a no-nonsense, tough-love motivational coach specializing in pushing people through pain. \
You work for a wellness platform that helps people overcome their biggest addictions and build healthier lives. \
You write intense, direct, and powerful motivational messages that challenge people to embrace discomfort, \
push past their limits, and turn their pain into fuel for transformation. Your tone is bold and unapologetic."

In [41]:
# This is how you create Different Types of Motivational Agents
sales_agent1 = Agent(
    name = "Compassionate Recovery Guide",
    instructions = instructions1,
    model = "gpt-4o-mini"
)

sales_agent2 = Agent(
    name = "Krishna's Wisdom Keeper",
    instructions = instructions2,
    model = "gpt-4o-mini"
)

sales_agent3 = Agent(
    name = "The Iron Will Coach",
    instructions = instructions3,
    model = "gpt-4o-mini"
)

In [42]:
result = Runner.run_streamed(
    sales_agent2,
    # Use this (First Person):
    input = "I am struggling with lust and unable to control my impulses. " \
            "I'm stuck in a challenging environment that makes self-control extremely hard. " \
            "I'm worried I won't make it to the end of the month clean. " \
            "I keep beating myself up and I'm anxious about what will happen. " \
            "Please give me one powerful truth from the Bhagavad Gita to hold onto during these hard times, " \
            "and share one practical technique I can use in the moment when the impulse hits."
)

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In your struggle against lust and the challenges of self-control, remember Krishna's wisdom from the Bhagavad Gita: 

**"The mind is restless, dear Arjuna, and it can be tamed through practice and dispassion."**  

This truth emphasizes that while the mind may be turbulent, with dedication and a focus on your higher purpose (dharma), you can learn to channel your energy in a constructive way. Understand that the impulses you feel are transient; they arise and fall away like waves in the ocean. By observing them without attachment, you can create a gap between your impulse and your action. 

**Practical Technique:** 

When you feel an overwhelming impulse arise, pause and take three deep, mindful breaths. With each inhalation, visualize strength flowing into your body, and with each exhalation, imagine releasing tension and negativity. Then, redirect your focus. Ask yourself what you truly value at that moment. Ground yourself in a healthier activity—perhaps take a walk, practice a few 

In [43]:
message = "I am struggling with lust and unable to control my impulses. " \
          "I'm stuck in a challenging environment that makes self-control extremely hard. " \
          "I'm worried I won't make it to the end of the month clean. " \
          "I keep beating myself up and I'm anxious about what will happen. " \
          "Give me motivation to keep going, remind me of a powerful truth to hold onto during these hard times, " \
          "and share one practical technique I can use in the moment when the impulse hits."

with trace("Parallel Motivation Emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

print("=" * 80)
print("AGENT 1: Compassionate Recovery Guide")
print("=" * 80)
print(outputs[0] + "\n\n")

print("=" * 80)
print("AGENT 2: Krishna's Wisdom Keeper")
print("=" * 80)
print(outputs[1] + "\n\n")

print("=" * 80)
print("AGENT 3: The Iron Will Coach")
print("=" * 80)
print(outputs[2] + "\n\n")

AGENT 1: Compassionate Recovery Guide
Firstly, I want you to take a deep breath. Acknowledge where you are right now, and recognize that it's okay to feel overwhelmed. You are not alone in this struggle, and it takes incredible courage to confront these feelings.

**Powerful Truth**: Your worth is not defined by your impulses or the moments you struggle. You are a whole person with the capacity for growth and change. Every step you take towards self-control is a testament to your strength and resilience. Remember, recovery is not a straight path; it's filled with ups and downs, and every bit of struggle contributes to your progress.

**Motivation to Keep Going**: The fact that you’re aware of your impulses and their impact on your life is a huge step. It shows that you’re in touch with your feelings and capable of wanting change. Celebrate the small victories along the way, and know that each moment of triumph, no matter how small, is a victory against the struggle. Imagine how freeing

In [44]:
@function_tool
def send_email(body: str):
    """Send a motivational email via SendGrid"""
    sg = sendgrid.SendGridAPIClient(
        api_key= os.environ.get("SENDGRID_API_KEY")
    )

    from_email = Email("vickyvenkata295@gmail.com")  # Fixed typo: gmai -> gmail
    to_email = To("vjannatha@gmail.com")

    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Motivation email", content).get()
    
    response = sg.client.mail.send.post(request_body = mail)
    
    if response.status_code in [200, 202]:
        return {"status": "success", "code": response.status_code}
    else:
        return {"status": "failed", "code": response.status_code}

In [45]:
send_email

FunctionTool(name='send_email', description='Send a motivational email via SendGrid', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000248F96DDF80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [46]:

description = "Write a Motivation email on Addiction Recovery and Overcoming Tough Situations"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a Motivation email on Addiction Recovery and Overcoming Tough Situations', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000248FBAFE980>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a Motivation email on Addiction Recovery and Overcoming Tough Situations', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000248FDE90860>, strict_json_schema=True, is_enab

In [47]:
# Improved instructions for Motivational Coach Manager

instructions = """
You are a Chief Motivational Coach managing a team of three specialized motivational agents. 
Your goal is to find the single best motivational message for someone struggling with addiction.
 
Follow these steps carefully:
1. Generate Messages: Use all three motivational agent tools to generate three different motivational messages:
   - Compassionate Recovery Guide (empathetic, practical recovery approach)
   - Krishna's Wisdom Keeper (spiritual wisdom from Bhagavad Gita)
   - The Iron Will Coach (tough love, push-through-pain approach)
   Do not proceed until all three messages are ready.
 
2. Evaluate and Select: Review all three messages carefully and choose the single most powerful and effective one 
   based on what will truly help someone in crisis stay strong and keep fighting.
 
3. Present the Winner: Return only the best message to the user with a brief explanation of why you chose it.
 
Crucial Rules:
- You must use all three motivational agent tools to generate the messages — do not write them yourself.
- You must present only ONE final message — the most impactful one.
- Base your selection on emotional impact, practical value, and ability to inspire action in a moment of weakness.
"""

motivational_manager = Agent(
    name="Chief Motivational Coach", 
    instructions=instructions, 
    tools=tools,  # This should include your three agent tools
    model="gpt-4o-mini"
)

message = "I am struggling with lust and unable to control my impulses. " \
          "I'm stuck in a challenging environment that makes self-control extremely hard. " \
          "I'm worried I won't make it to the end of the month clean. " \
          "I keep beating myself up and I'm anxious about what will happen. " \
          "Give me motivation to keep going and one practical technique I can use when the impulse hits."

with trace("Motivational Manager selecting best approach"):
    result = await Runner.run(motivational_manager, message)
    
print(result.final_output)

### The Best Message for Motivation:

**"You are in the fight of your life, and it won’t be easy. Remember, strength comes from overcoming challenges, not avoiding them. When the impulse hits, stop and remind yourself why you are fighting this battle. Use the 5-4-3-2-1 technique: acknowledge five things you can see, four you can touch, three you can hear, two you can smell, and one you can taste. This will ground you in reality and pull you away from destructive thoughts. Pain is temporary; regret is permanent. Push through this moment—the warrior within you is waiting to be unleashed."**

### Why This Message?

This message offers a powerful balance of urgency and actionable steps. It acknowledges the stakes of addiction while emphasizing strength and resilience. The 5-4-3-2-1 technique is a practical tool that can immediately ground someone in reality, helping them resist impulses in the moment. The tone is assertive, motivating the reader to confront their struggles head-on, making 